In [1]:
import numpy as np

# 25 espacios conectados con lineas verticales, horizontales y diagonales
# 4 fichas negras y 4 piezas rojas
# Si un jugador forma una linea de 4 fichas Gana la partida (Horizontal, Diagonal y Vertical)
# Si un jugador forma un cuadrado Gana la partida
# Una vez asignadas las 4 piezas el jugador puede seleccionar una y moverla a una casilla sin asignacion

#link#
#los jugadores sortean quien comienza, por defecto comienza el negro y se alternan entre sí
#############################################################primera fase#############################################################################################
#En la primera parte los jugadores ponen sus marcadores, y cada marcador ocupa un campo, para el final de esta fase hay 8 fichas en el tablero, 4 negras y 4 rojas
#Es posible ganar en la primera fase, si nadie gana, se pasa a la segunda fase

#############################################################segunda fase#############################################################################################
#No se puede saltar turnos
#en cada turno se puede mover las fichas en un campo adyacente vacio, ya sea vertical, horizontal o diagonal

##############################################################objective###############################################################################################
#organizar las fichas hasta formar un Teeko, existen 44 Teekos distintos, las mismas se dividen en verticales, horizontales , diagonales y cuadradas


move_dict = {
  "A": 0,
  "B": 1,
  "C": 2,
  "D": 3,
  "E": 4,
}

class Teeko:
    def __init__(self, player1, player2):
        self.player1 = player1
        self.player2 = player2
        self.board = np.zeros((5,5))
        
    
    def move_translator(self, selected, move):       #A5, B6 etc.
        yPos = selected[0]
        xPos = selected[1]
        return (move_dict[yPos], xPos)

    def make_a_move(self, selected, move = None):       # selected recibe posiciones reales tupla, move recibe la dirección  # move
        if move is not None:
            self.board[selected[0]][selected[1]] = 0
            y, x = selected
            valid_move = True
            for pos in move:
                if pos == "u" and y > 0:
                    y -= 1
                elif pos == "d" and y < 4:
                    y += 1
                elif pos == "r" and x < 4:
                    x += 1 
                elif pos == "l" and x > 0: 
                    x -= 1
                else: 
                    valid_move = False      
            if valid_move:
                selected = (y, x)
            else: 
                print("Movimiento no válido, vuelva a intentarlo")
        self.board[selected[0]][selected[1]] = self.current_player        
            
        return valid_move
        

    def check_winner(move):
        # Busca si el ultimo movimiento del jugador es uno ganador
        # Si el jugador forma un cuadrado se le devuelve como ganador
        # Si el jugador forma una linea horizontal se le devuleve como ganador
        # Si el jugador forma una linea vertical se le devuelve como ganador 
        # Si el jugadro forma una linea diagonal se le devuelve como ganador
        return 0

